<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/portfolios/optimal_yahoo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from cvxopt import matrix as Matrix
from cvxopt.solvers import qp as Solver
from cvxopt.solvers import options as SolverOptions
SolverOptions['show_progress'] = False

def optimal_s(rs, rb, means, cov, raver):
    n = len(means)
    Q = np.zeros((n + 2, n + 2))
    Q[2:, 2:] = raver * cov
    Q = Matrix(Q, tc="d")                               # quadratic form that penalizes variance
    p = np.array([-rs, rb] + list(-np.array(means)))
    p = Matrix(p, (len(p), 1), tc="d")                  # coefficients of objective function
    g = np.array([1, -1] + n * [1])
    a = np.zeros((2, n + 2))
    a[0, 0] = a[1, 1] = -1
    G = Matrix(np.vstack([a, g]))                       # coefficients of constraints
    h = np.array(2 * [0] + [1])
    h = Matrix(h, (len(h), 1), tc="d")                  # right-hand side of constraints
    sol = Solver(Q, p, G, h)
    return sol['x'] if sol['status']=='optimal' else None

def optimal_ns(rs, rb, means, cov, raver) :
    n = len(means)
    Q = np.zeros((n+2, n+2))
    Q[2:,2:] = raver*cov
    Q = Matrix(Q, tc='d')                                             # quadratic form that penalizes variance
    p = np.array([-rs, rb] + list(-np.array(means)))
    p = Matrix(p, (len(p), 1), tc='d')                                # coefficients of objective function
    g = np.array([1, -1] + n*[1])
    G = Matrix(np.vstack([-np.identity(n+2), g]), tc='d')             # coefficients of constraints
    h = np.array((n+2)*[0] + [1])
    h = Matrix(h, (len(h), 1), tc='d')                                # right-hand side of constraints
    sol = Solver(Q, p, G, h)
    return sol['x'] if sol['status']=='optimal' else None


In [ ]:
from pandas_datareader import DataReader as pdr
tickers = ['aple', 'nvda', 'amd']
df = pdr(tickers, "yahoo", 1970)
df = df["Adj Close"].resample("M").last().pct_change().dropna().iloc[:-1]
df.head()

Symbols,aple,nvda,amd
Date,,,
2015-06-30,0.028191,-0.091279,0.052632
2015-07-31,-0.033288,-0.007956,-0.195833
2015-08-31,-0.017914,0.131558,-0.062176
2015-09-30,0.054143,0.096530,-0.049724
2015-10-31,0.067416,0.150913,0.232558


In [ ]:
means = df.mean().to_numpy()
cov = df.cov().to_numpy()
rs = 0.02                             # example savings rate
rb = 0.06                             # example borrowing rate
raver = 2                             # example risk aversion
rs_monthly = (1 + rs) ** (1 / 12) - 1 # monthly savings rate
rb_monthly = (1 + rb) ** (1 / 12) - 1 # monthly borrowing rate

sol = optimal_ns(rs, rb, means, cov, raver)

if sol:
    sol = list(sol)
    print('Without Short Sales:')
    print(f'optimal amount to save is {sol[0]:.1%}')
    print(f'optimal amount to borrow is {sol[1]:.1%}')
    for i in range(len(means)) :
        print(f'optimal investment in asset {tickers[i]} is {sol[i+2]:.1%}')

Without Short Sales:
optimal amount to save is 0.0%
optimal amount to borrow is 0.0%
optimal investment in asset aple is 0.0%
optimal investment in asset nvda is 55.7%
optimal investment in asset amd is 44.3%


In [ ]:
means = df.mean().to_numpy()
cov = df.cov().to_numpy()
rs = 0.02                             # example savings rate
rb = 0.06                             # example borrowing rate
raver = 2                             # example risk aversion
rs_monthly = (1 + rs) ** (1 / 12) - 1 # monthly savings rate
rb_monthly = (1 + rb) ** (1 / 12) - 1 # monthly borrowing rate

sol = optimal_s(rs, rb, means, cov, raver)

if sol:
    sol = list(sol)
    print('Without Short Sales:')
    print(f'optimal amount to save is {sol[0]:.1%}')
    print(f'optimal amount to borrow is {sol[1]:.1%}')
    for i in range(len(means)) :
        print(f'optimal investment in asset {tickers[i]} is {sol[i+2]:.1%}')

Without Short Sales:
optimal amount to save is 158.4%
optimal amount to borrow is 0.0%
optimal investment in asset aple is -198.0%
optimal investment in asset nvda is 77.0%
optimal investment in asset amd is 62.7%
